# create_full_service_dfs
This notebook:
1. Loads the npy distance matrix file (made for the galveston moody gardens + indoor + outdoor points) and converts it into a csv file --> saves the csv file in the data folder 
2. Concatonates the galveston_indoor_pts and galveston_outdoor_pts dataframes into one full dataframe will all service locations
3. Performs data cleaning on the merged service locations df --> saves the csv in the data folder   

In [57]:
import pandas as pd
import numpy as np

# Part 1. Load and Save the Distance Matrix for the indoor + outdoor points

In [58]:
#load the distance matrix file, it is a numpy array
dist_matrix_array = np.load("../data/distance_matrix_new.npy") 
dist_matrix_array

array([[    0. ,  5143.2,  8990.2, ..., 11893.4, 15070.8, 14999.4],
       [ 5286.6,     0. , 12326. , ...,  9672. , 12849.4, 12778.1],
       [ 8678.1, 11909.4,     0. , ..., 17552.7, 20730. , 20658.7],
       ...,
       [12035. ,  9690.9, 17580.9, ...,     0. ,  3615.7,  3544.3],
       [15209. , 12864.9, 20754.8, ...,  3434.3,     0. ,   172.2],
       [15194.5, 12850.4, 20740.3, ...,  3419.8,   171.6,     0. ]])

In [59]:
#convert the ditance matrix to a dataframe
dist_matrix_df=pd.DataFrame(dist_matrix_array)
#name columns
dist_matrix_df.columns = [str(i) for i in range(256)]
dist_matrix_df

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,0.0,5143.2,8990.2,6368.1,13815.7,10253.6,9602.4,9632.7,9629.0,9691.5,...,5572.7,7771.3,1849.6,1672.0,13248.8,6031.2,8261.5,11893.4,15070.8,14999.4
1,5286.6,0.0,12326.0,9623.2,15375.9,13589.5,12938.2,12968.5,12964.8,13027.3,...,3351.3,10925.6,5359.4,3439.3,16584.6,3763.6,6040.2,9672.0,12849.4,12778.1
2,8678.1,11909.4,0.0,2614.5,5088.0,1389.6,7453.6,905.0,901.3,967.6,...,11232.0,1136.5,7011.3,8690.0,11100.0,11690.5,13920.8,17552.7,20730.0,20658.7
3,6410.6,9642.0,2574.4,0.0,7399.9,3701.5,5968.7,3216.9,3213.2,3275.7,...,9453.6,2756.7,4743.8,6422.5,9615.2,9912.2,12142.5,15774.3,18951.7,18880.4
4,14122.0,17353.3,5111.7,7424.9,0.0,5271.7,11966.6,5250.3,5246.6,5542.6,...,15424.6,6248.1,12243.7,14133.9,15613.0,15883.2,18113.5,21745.3,24922.7,24851.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,6173.0,3785.7,11718.9,9889.1,15826.3,14392.5,13741.2,12845.9,12842.2,12904.8,...,1109.9,10974.9,6245.8,4354.0,17387.6,0.0,2555.5,6187.4,9364.8,9293.4
252,8419.0,6074.9,13964.8,12135.1,18072.3,16638.5,15987.2,15091.9,15088.2,15150.8,...,3355.9,13220.9,8491.8,6600.0,19633.6,2513.0,0.0,3966.8,7144.2,7072.9
253,12035.0,9690.9,17580.9,15751.1,21688.3,20254.5,19603.2,18707.9,18704.3,18766.8,...,6971.9,16836.9,12107.9,10216.0,23249.6,6129.0,3893.7,0.0,3615.7,3544.3
254,15209.0,12864.9,20754.8,18925.1,24862.3,23428.5,22777.2,21881.9,21878.2,21940.8,...,10145.9,20010.9,15281.8,13390.0,26423.6,9303.0,7067.6,3434.3,0.0,172.2


In [60]:
#save to csv
dist_matrix_df.to_csv("../data/indoor_outdoor_distances_galv.csv", index=False)

# Part 2. Create full tote service locations df by concatenating the indoor and outdoor points dfs, and doing light data cleaning

-Import the Galveston indoor and outdoor points: galveston_indoor_pts and galveston_outdoor_pts
-Merge information from the old FUE dataframe
-Perform data cleaning

1. Merge columns "Daily_Pickup_Totes" and "Weekly_Dropoff_Totes" from the original dataframe to the indoor points dataframe
2. Add the "Moody Gardens" location to the the top of the indoor locations dataframe
3. Add a column "location_type" = "indoor" to every point in the indoor dataframe 
4. Add a column "pickup_type" = "Truck" to every point in the indoor dataframe

5. Add a "Daily_Pickup_Totes" of 1.0 to every outdoor point
6. Add a column "location_type" = "outdoor" to every point in the outdoor dataframe 
7. Capitalize the "longitude" and "latitude" columns in the outdoor df

8. Concatonate the indoor and outdoor points into a single dataframe
9. Set "location_type" for Moody Gardens to "depot"
10. Fill in missing values for "Weekly_Pickup_Totes" and "Weekly_Dropoff_Totes" with 1.0 each
11. Convert "Daily_Pickup_Totes" and "Weekly_Dropoff_Totes" columns to int

12. reset index and save dataframe

In [61]:
# Load the indoor pts df and the original FUE df
indoor = pd.read_csv("../data/galveston_indoor_pts.csv")
old_galveston = pd.read_csv("../data/FUE_Galveston.csv")

In [62]:
#1. merge the indoor pts df with the columns "Weekly_Dropoff_Totes" 
#and "Daily_Pickup_Totes" from the original df

galveston_sub = old_galveston.loc[:, ["Name", "Weekly_Dropoff_Totes", "Daily_Pickup_Totes"]]
indoor = pd.merge(indoor, 
                        galveston_sub, 
                        on = "Name", 
                        how = 'left')

In [63]:
# 2. add the "Moody Gardens" location to the top of the indoor dataframe

#extract the Moody Gardens columns from the old dataframe
moody_gardens = old_galveston.loc[(old_galveston.loc[:, "Name"]==
                               "Moody Gardens"), (
                            ["Name", "Longitude", "Latitude", "Daily_Pickup_Totes", "Weekly_Dropoff_Totes"])]

#concatonate the column on top of the indoor points df
moody_plus_indoor = pd.concat([moody_gardens, indoor])

In [64]:
# 3. add a column "location_type" = "indoor" to every point in the indoor dataframe 
moody_plus_indoor.loc[:, "location_type"] = "indoor"

In [65]:
# 4. dd a column "pickup_type" = "Truck" to every point in the indoor dataframe
moody_plus_indoor.loc[:, "pickup_type"] = "Truck"

In [66]:
#Load outdoor pts df
outdoor = pd.read_csv("../data/galveston_outdoor_pts.csv")

In [67]:
# 5. add a daily pickup value of 1.0 and weekly dropoff value to 0.0 to every outdoor point
outdoor.loc[:, "Daily_Pickup_Totes"] = 1.0
outdoor.loc[:, "Weekly_Dropoff_Totes"] = 0.0

In [68]:
# 6. add a column "location_type" = "outdoor" to every point in the outdoor dataframe 
outdoor.loc[:, "location_type"] = "outdoor"

In [69]:
# 7. capitalize the "longitude" and "latitude" columns in the outdoor df
outdoor = outdoor.rename(columns={"longitude": "Longitude", "latitude": "Latitude"})

In [70]:
# 8. concatonate the indoor and outdoor points into a single dataframe
full_service_locations = pd.concat([moody_plus_indoor, outdoor])
full_service_locations.head()

,Name,user_ratings_total,category,rating,Latitude,Longitude,weekly_high,weekly_low,best_weekly_estimate,Address,geometry,Adoption,Adoption Guess,Visited,Influence,Weekly_Dropoff_Totes,Daily_Pickup_Totes,location_type,pickup_type,id
0,Sugar Bean,476.0,cafe,4.7,29.261029,-94.870418,200,50,200,"11 Evia Main, Galveston",POINT (-94.8704183 29.2610286),Curious,High,No,High,1.0,1.0,indoor,Truck,NaN
1,MAC-IES,73.0,restaurant,5.0,29.298912,-94.791385,1500,1050,1500,"1110 23rd Street, Galveston",POINT (-94.7913851 29.2989118),Curious,High,No,Low,1.0,1.0,indoor,Truck,NaN
2,Good Dough,68.0,bakery,4.6,29.290612,-94.808531,70,35,70,"1508 39th Street, Galveston",POINT (-94.808531 29.2906119),Curious,High,No,Med,1.0,1.0,indoor,Truck,NaN
3,Porch Cafe,332.0,restaurant,4.1,29.319581,-94.750192,300,NaN,100,"1625 East Beach Drive #2A, Galveston",POINT (-94.7501916 29.3195806),Curious,High,No,Low,1.0,1.0,indoor,Truck,NaN
4,Coastal Grill,814.0,restaurant,4.6,29.308158,-94.789455,500,100,500,"1827 The Strand, Galveston",POINT (-94.789455 29.308158),Skeptical,High,No,Med,1.0,1.0,indoor,Truck,NaN


In [71]:
# 9. sets "location_type" = "depot" for "Moody Garden" 
full_service_locations.loc[(full_service_locations.loc[:, "Name"]== "Moody Gardens"),
                            "location_type"] = "depot"
full_service_locations.head()

,Name,user_ratings_total,category,rating,Latitude,Longitude,weekly_high,weekly_low,best_weekly_estimate,Address,geometry,Adoption,Adoption Guess,Visited,Influence,Weekly_Dropoff_Totes,Daily_Pickup_Totes,location_type,pickup_type,id
0,Sugar Bean,476.0,cafe,4.7,29.261029,-94.870418,200,50,200,"11 Evia Main, Galveston",POINT (-94.8704183 29.2610286),Curious,High,No,High,1.0,1.0,indoor,Truck,NaN
1,MAC-IES,73.0,restaurant,5.0,29.298912,-94.791385,1500,1050,1500,"1110 23rd Street, Galveston",POINT (-94.7913851 29.2989118),Curious,High,No,Low,1.0,1.0,indoor,Truck,NaN
2,Good Dough,68.0,bakery,4.6,29.290612,-94.808531,70,35,70,"1508 39th Street, Galveston",POINT (-94.808531 29.2906119),Curious,High,No,Med,1.0,1.0,indoor,Truck,NaN
3,Porch Cafe,332.0,restaurant,4.1,29.319581,-94.750192,300,NaN,100,"1625 East Beach Drive #2A, Galveston",POINT (-94.7501916 29.3195806),Curious,High,No,Low,1.0,1.0,indoor,Truck,NaN
4,Coastal Grill,814.0,restaurant,4.6,29.308158,-94.789455,500,100,500,"1827 The Strand, Galveston",POINT (-94.789455 29.308158),Skeptical,High,No,Med,1.0,1.0,indoor,Truck,NaN


In [72]:
# 10. fill in missing values for pickup and dropoff totes
full_service_locations.loc[full_service_locations.loc[:, "Daily_Pickup_Totes"].isna(), "Daily_Pickup_Totes"] = 1.0
full_service_locations.loc[full_service_locations.loc[:, "Weekly_Dropoff_Totes"].isna(), "Weekly_Dropoff_Totes"] = 1.0

In [73]:
full_service_locations.loc[:, "Daily_Pickup_Totes"] = full_service_locations.loc[:, "Daily_Pickup_Totes"].astype(int)
full_service_locations.loc[:, "Weekly_Dropoff_Totes"] = full_service_locations.loc[:, "Weekly_Dropoff_Totes"].astype(int)

/var/folders/4b/3cdcp3h14kl45v92kx9_styr0000gn/T/ipykernel_75618/2976372323.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  full_service_locations.loc[:, "Daily_Pickup_Totes"] = full_service_locations.loc[:, "Daily_Pickup_Totes"].astype(int)
/var/folders/4b/3cdcp3h14kl45v92kx9_styr0000gn/T/ipykernel_75618/2976372323.py:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  full_service_locations.loc[:, "Weekly_Dropoff_Totes"] = full_service_locations.loc[:, "Weekly_Dropoff_Totes"].astype(int)


Save the resulting dataframe.

In [74]:
#reset index and check
full_service_locations.reset_index(inplace=True, drop=True)
full_service_locations

,Name,user_ratings_total,category,rating,Latitude,Longitude,weekly_high,weekly_low,best_weekly_estimate,Address,geometry,Adoption,Adoption Guess,Visited,Influence,Weekly_Dropoff_Totes,Daily_Pickup_Totes,location_type,pickup_type,id
0,Sugar Bean,476.0,cafe,4.7,29.261029,-94.870418,200,50,200,"11 Evia Main, Galveston",POINT (-94.8704183 29.2610286),Curious,High,No,High,1,1,indoor,Truck,NaN
1,MAC-IES,73.0,restaurant,5.0,29.298912,-94.791385,1500,1050,1500,"1110 23rd Street, Galveston",POINT (-94.7913851 29.2989118),Curious,High,No,Low,1,1,indoor,Truck,NaN
2,Good Dough,68.0,bakery,4.6,29.290612,-94.808531,70,35,70,"1508 39th Street, Galveston",POINT (-94.808531 29.2906119),Curious,High,No,Med,1,1,indoor,Truck,NaN
3,Porch Cafe,332.0,restaurant,4.1,29.319581,-94.750192,300,NaN,100,"1625 East Beach Drive #2A, Galveston",POINT (-94.7501916 29.3195806),Curious,High,No,Low,1,1,indoor,Truck,NaN
4,Coastal Grill,814.0,restaurant,4.6,29.308158,-94.789455,500,100,500,"1827 The Strand, Galveston",POINT (-94.789455 29.308158),Skeptical,High,No,Med,1,1,indoor,Truck,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,NaN,NaN,NaN,NaN,29.238725,-94.878118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,outdoor,Truck,3030.0
251,NaN,NaN,NaN,NaN,29.227723,-94.896043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,outdoor,Truck,3031.0
252,NaN,NaN,NaN,NaN,29.210431,-94.927823,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,outdoor,Truck,3032.0
253,NaN,NaN,NaN,NaN,29.193525,-94.954036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,outdoor,Truck,3033.0


In [75]:
#save to csv file
full_service_locations.to_csv("../data/indoor_outdoor_pts_galv.csv", index=False)